In [2]:
import random
from torchvision.ops import box_iou
import os
import json
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import numpy as np
from tqdm import tqdm
from torchvision.transforms import functional as F
os.environ['KMP_DUPLICATE_LIB_OK']='True'


class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target


class ToTensor:
    def __call__(self, image, target):
        return F.to_tensor(image), target

class RandomHorizontalFlip:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            _, width = image.shape[-1], image.shape[-2] if isinstance(image, torch.Tensor) else image.size
            image = F.hflip(image)
            boxes = target["boxes"].clone()
            boxes[:, [0, 2]] = width - boxes[:, [2, 0]]  # 翻转x坐标
            # 确保坐标顺序正确
            xmin = torch.min(boxes[:, 0], boxes[:, 2])
            xmax = torch.max(boxes[:, 0], boxes[:, 2])
            ymin = boxes[:, 1]
            ymax = boxes[:, 3]
            target["boxes"] = torch.stack((xmin, ymin, xmax, ymax), dim=1)
        return image, target

class DocumentTamperDataset(Dataset):
    def __init__(self, image_dir, annotation_path, DEBUG_SUBSET_SIZE, transforms=None, debug_mode=False):
        self.image_dir = image_dir
        self.transforms = transforms
        with open(annotation_path) as f:
            self.annotations = json.load(f)
        if debug_mode:
            self.annotations = self.annotations[:DEBUG_SUBSET_SIZE]
        self.id_to_anns = {item['id']: item['region'] for item in self.annotations}
        self.ids = list(self.id_to_anns.keys())

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        img_path = os.path.join(self.image_dir, img_id)
        img = Image.open(img_path).convert("RGB")
        regions = self.id_to_anns[img_id]

        boxes = []
        for region in regions:
            if len(region) != 4:
                continue  # 跳过不符合要求的区域
            coords = list(map(float, region))
            x1, y1, x2, y2 = coords
            xmin, xmax = sorted([x1, x2])
            ymin, ymax = sorted([y1, y2])
            if xmin >= xmax or ymin >= ymax:
                print(f"发现无效框，已过滤: {coords} in {img_id}")
                continue
            boxes.append([xmin, ymin, xmax, ymax])

        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
        else:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((len(boxes),), dtype=torch.int64)  
        target = {
            "boxes": boxes,
            "labels": torch.ones((len(boxes),), dtype=torch.int64), 
            "image_id": torch.tensor([idx])
            }

        if self.transforms:
            img, target = self.transforms(img, target)
            
        return img, target

    def __len__(self):
        return len(self.ids)


def get_transform(train=True):
    transforms = []
    if train:
        transforms.append(RandomHorizontalFlip(0.5))
    transforms.append(ToTensor())
    return Compose(transforms)

# 创建模型
def create_model(num_classes=2):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

# 训练函数
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    
    total_loss = 0
    progress_bar = tqdm(data_loader, 
                       desc=f"Epoch {epoch} Training",
                       leave=True,  
                       dynamic_ncols=True,  
                       bar_format="{l_bar}{bar:20}{r_bar}")
    
    for images, targets in progress_bar:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        
        total_loss += losses.item()
        progress_bar.set_postfix(loss=losses.item())
    
    return total_loss / len(data_loader)

def calculate_metrics(pred_boxes, true_boxes, iou_threshold=0.5):
    """
    计算单个样本的TP、FP、FN
    """
    if len(pred_boxes) == 0:
        return 0, 0, len(true_boxes)
    
    if len(true_boxes) == 0:
        return 0, len(pred_boxes), 0

    iou_matrix = box_iou(pred_boxes, true_boxes)

    matched_true = set()
    matched_pred = set()

    for true_idx in range(len(true_boxes)):
        best_iou = iou_threshold
        best_pred = -1
        for pred_idx in range(len(pred_boxes)):
            iou = iou_matrix[pred_idx, true_idx]
            if iou > best_iou:
                best_iou = iou
                best_pred = pred_idx
        if best_pred != -1 and best_pred not in matched_pred:
            matched_true.add(true_idx)
            matched_pred.add(best_pred)
    
    tp = len(matched_true)
    fp = len(pred_boxes) - len(matched_pred)
    fn = len(true_boxes) - len(matched_true)
    
    return tp, fp, fn

def evaluate(model, data_loader, device):
    model.eval()
    
    total_tp = 0
    total_fp = 0
    total_fn = 0
    
    progress_bar = tqdm(data_loader, 
                       desc="Validating",
                       leave=True,
                       dynamic_ncols=True,
                       bar_format="{l_bar}{bar:20}{r_bar}")
    
    with torch.no_grad():
        for images, targets in progress_bar:
            images = list(img.to(device) for img in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            
            predictions = model(images)
            
            for pred, true in zip(predictions, targets):
                keep = pred['scores'] > 0.5
                pred_boxes = pred['boxes'][keep].cpu()
                true_boxes = true['boxes'].cpu()
                
                tp, fp, fn = calculate_metrics(pred_boxes, true_boxes)
                
                total_tp += tp
                total_fp += fp
                total_fn += fn

    # 转换为浮点数（不再使用.item()）    precision = total_tp / (total_tp + total_fp + 1e-10)
    recall = total_tp / (total_tp + total_fn + 1e-10)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-10)
    
    return {
        "Micro_Prec": float(precision),
        "Micro_Recall": float(recall),
        "Micro_F1": float(f1)
    }

import os
import pandas as pd
import matplotlib.pyplot as plt
from torchvision.utils import save_image

def main():
    # 超参数配置
    BATCH_SIZE = 4
    NUM_EPOCHS = 15
    LR = 0.005
    VAL_SPLIT = 0.2
    DEBUG_MODE = False
    DEBUG_SUBSET_SIZE = 10
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 数据集加载
    full_dataset = DocumentTamperDataset(
        image_dir="train/images",
        annotation_path="train/label_train.json",
        DEBUG_SUBSET_SIZE=DEBUG_SUBSET_SIZE,
        transforms=get_transform(train=True),
        debug_mode=DEBUG_MODE
    )
    
    # 数据划分
    dataset_size = len(full_dataset)
    val_size = int(VAL_SPLIT * dataset_size)
    train_size = dataset_size - val_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))
    val_dataset.dataset.transforms = get_transform(train=False)
    
    # 数据加载器
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
    
    # 初始化模型
    model = create_model(num_classes=2)
    model.to(device)
    
    # 优化器配置
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=LR, momentum=0.9, weight_decay=0.0005)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    metrics_history = []
    
    # 训练循环
    best_f1 = 0
    for epoch in range(NUM_EPOCHS):
        train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
        val_metrics = evaluate(model, val_loader, device)
        lr_scheduler.step()
        
        print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Validation Metrics: Precision={val_metrics['Micro_Prec']:.4f}, Recall={val_metrics['Micro_Recall']:.4f}, F1={val_metrics['Micro_F1']:.4f}")
        
        # 保存当前epoch的指标
        metrics_history.append({
            "Epoch": epoch + 1,
            "Train_Loss": train_loss,
            "Precision": val_metrics["Micro_Prec"],
            "Recall": val_metrics["Micro_Recall"],
            "F1": val_metrics["Micro_F1"]
        })
        
        if val_metrics['Micro_F1'] > best_f1:
            best_f1 = val_metrics['Micro_F1']
            torch.save(model.state_dict(), "best_model.pth")
            print("↦ 保存新最佳模型")
        

 
    # 最终评估
    model.load_state_dict(torch.load("best_model.pth"))
    final_metrics = evaluate(model, val_loader, device)
    print("\n" + "="*50)
    print(f"最终评估结果: Precision={final_metrics['Micro_Prec']:.4f}, Recall={final_metrics['Micro_Recall']:.4f}, F1={final_metrics['Micro_F1']:.4f}")
    print("="*50)

    # 保存所有指标到CSV
    save_metrics_to_csv(metrics_history)

def save_metrics_to_csv(metrics_history, filename="training_metrics.csv"):
    """保存训练过程中的指标到CSV文件"""
    df = pd.DataFrame(metrics_history)
    df.to_csv(filename, index=False)
    print(f"Training metrics saved to {filename}")



if __name__ == "__main__":
    main()


Epoch 0 Training:   0%|                    | 7/2600 [00:02<17:06,  2.53it/s, loss=0.119]

发现无效框，已过滤: [152.0, 20.0, 152.0, 91.0] in train_13271.jpg


Epoch 0 Training:   2%|▎                   | 41/2600 [00:14<13:57,  3.05it/s, loss=0.0889]

发现无效框，已过滤: [203.0, 19.0, 203.0, 89.0] in train_13554.jpg


Epoch 0 Training:  24%|████▊               | 623/2600 [03:44<11:56,  2.76it/s, loss=0.0748] 

...


Epoch 15/15
Train Loss: 0.0242
Validation Metrics: Precision=0.6318, Recall=0.8240, F1=0.7152


Validating:  84%|████████████████▊   | 547/650 [01:37<00:17,  6.05it/s]

发现无效框，已过滤: [457.0, 60.0, 457.0, 91.0] in train_13338.jpg


Validating: 100%|████████████████████| 650/650 [01:55<00:00,  5.63it/s]


最终评估结果: Precision=0.6689, Recall=0.8005, F1=0.7288
Training metrics saved to training_metrics.csv
